<a href="https://colab.research.google.com/github/zzhining/ml_basic/blob/main/TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 12.3 MB/s 
     |████████████████████████████████| 465 kB 58.9 MB/s 


# One-hot Encoding

In [4]:
from konlpy.tag import Okt
tokenizer = Okt()  
tokens = tokenizer.morphs("한국의 수도는 서울이다")

print(tokens)

['한국', '의', '수도', '는', '서울', '이다']


In [5]:
# 단어-인덱스 딕셔너리
word_to_index = {}

# 토큰을 인덱스로 변환
for token in tokens:
    if token not in word_to_index.keys():
        word_to_index[token] = len(word_to_index)
        
print(word_to_index)


{'한국': 0, '의': 1, '수도': 2, '는': 3, '서울': 4, '이다': 5}


In [6]:
# 원핫인코딩으로 변환
def convert_ohe(word, word_to_index):
    
    # 벡터를 단어의 개수만큼 0으로 초기화
    vector = [0]*(len(word_to_index))
    
    # 단어의 인덱스 위치에 1 설정
    vector[word_to_index[word]] = 1
    
    return vector

In [7]:
convert_ohe("서울", word_to_index)


[0, 0, 0, 0, 1, 0]

<br>
<br>

# Bag-of-Words

In [8]:
tokenizer = Okt()  
text = "인공지능은 사람의 지능을 기계에 구현하였다"
tokens = tokenizer.morphs(text)

print(tokens)


['인공', '지능', '은', '사람', '의', '지능', '을', '기계', '에', '구현', '하였다']


In [9]:
# 단어-인덱스 딕셔너리
word_to_index = {}

# 토큰을 인덱스로 변환
for token in tokens:
    if token not in word_to_index.keys():
        word_to_index[token] = len(word_to_index)
        
print(word_to_index)


{'인공': 0, '지능': 1, '은': 2, '사람': 3, '의': 4, '을': 5, '기계': 6, '에': 7, '구현': 8, '하였다': 9}


In [10]:
# BoW로 변환
def convert_bow(sentence, word_to_index):
    
    # 벡터를 단어의 개수만큼 0으로 초기화
    vector = [0]*(len(word_to_index))

    # 문장을 토큰으로 분리
    tokenizer = Okt()
    tokens = tokenizer.morphs(sentence)
    
    # 단어의 인덱스 위치에 1 설정
    for token in tokens:
        if token in word_to_index.keys():
            vector[word_to_index[token]] += 1
    
    return vector

In [11]:
convert_bow("인공지능은 사람의 지능을 기계에 구현하였다", word_to_index)


[1, 2, 1, 1, 1, 1, 1, 1, 1, 1]

In [12]:
convert_bow("인공지능은 기계의 지능을 말한다", word_to_index)


[1, 2, 1, 0, 1, 1, 1, 0, 0, 0]

<br>
<br>

# CountVectorizer

In [13]:
# 토큰을 문자열로 변환
sentence = " ".join(tokens)

print(sentence)


인공 지능 은 사람 의 지능 을 기계 에 구현 하였다


In [14]:
# CountVectorizer의 입력에 맞게 배열로 변경
sentences = []
sentences.append(sentence)

print(sentences)


['인공 지능 은 사람 의 지능 을 기계 에 구현 하였다']


In [15]:
from sklearn.feature_extraction.text import CountVectorizer

# 1글자도 인식이 되도록 토큰 패턴 변경
cv = CountVectorizer(token_pattern = r"(?u)\b\w+\b")
cv.fit(sentences)

print(cv.vocabulary_)


{'인공': 7, '지능': 8, '은': 4, '사람': 2, '의': 6, '을': 5, '기계': 1, '에': 3, '구현': 0, '하였다': 9}


In [16]:
# CountVectorizer로 변환
def convert_cv(sentence, cv):
    
    # 문장을 토큰으로 분리
    tokenizer = Okt()
    tokens = tokenizer.morphs(sentence)
    
    # 토큰을 문자열로 변환
    sentence = " ".join(tokens)
    
    # CountVectorizer의 입력에 맞게 배열로 변경
    sentences = []
    sentences.append(sentence)
    
    # 벡터 변환
    vector = cv.transform(sentences).toarray()    
    
    return vector

In [17]:
convert_cv("인공지능은 사람의 지능을 기계에 구현하였다", cv)


array([[1, 1, 1, 1, 1, 1, 1, 1, 2, 1]])

In [18]:
convert_cv("인공지능은 기계의 지능을 말한다", cv)


array([[0, 1, 0, 0, 1, 1, 1, 1, 2, 0]])

<br>
<br>

# TF-IDF

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

documents = [
    "I like animals",
    "I like food",
    "I hate math",
    "I want to study math",
]

# TF-IDF로 벡터화
# 1글자도 인식이 되도록 토큰 패턴 변경
tf_idf = TfidfVectorizer(token_pattern = r"(?u)\b\w+\b")
tf_idf.fit(documents)

print(tf_idf.vocabulary_)

{'i': 3, 'like': 4, 'animals': 0, 'food': 1, 'hate': 2, 'math': 5, 'want': 8, 'to': 7, 'study': 6}


In [20]:
# 다른 문서에도 많이 나온 단어는 낮은 수치
tf_idf.transform(["I like animals"]).toarray()


array([[0.72664149, 0.        , 0.        , 0.37919167, 0.5728925 ,
        0.        , 0.        , 0.        , 0.        ]])

In [21]:
# 같은 문서에 많이 나온 단어는 높은 수치
tf_idf.transform(["I like animals and love animals"]).toarray()


array([[0.90406978, 0.        , 0.        , 0.23589056, 0.3563895 ,
        0.        , 0.        , 0.        , 0.        ]])

<br>
<br>

# TF-IDF로 유사도 비교

- https://www.kaggle.com/fajim123/movies-metadata
- https://www.kaggle.com/rounakbanik/movie-recommender-systems/comments
- [Easiest way to download kaggle data in Google Colab](https://www.kaggle.com/general/74235)

In [36]:
import pandas as pd

# 영화 데이터셋 로드
data = pd.read_csv("movies_metadata.csv", low_memory=False)

len(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [37]:
# 첫 번째 데이터 출력
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [38]:
len(data['title'].unique())

42278

In [39]:
# 첫 번째 데이터의 overview 출력
# 영화에 대한 설명
data.head(1)["overview"][0]


"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

In [40]:
# 데이터의 일부만 사용
data = data.head(10000)

# Null인 항목의 개수
data["overview"].isnull().sum()


29

In [41]:
# Null인 항목을 빈 값으로 대체
data["overview"] = data["overview"].fillna("")

In [42]:
# Null인 항목이 없어야 TfidfVectorizer() 가능
data["overview"].isnull().sum()


0

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 변환
tf_idf = TfidfVectorizer(stop_words="english")
tf_idf_matrix = tf_idf.fit_transform(data["overview"])

# 데이터의 개수 : 10000
# 단어의 개수 : 32350
print(tf_idf_matrix.shape)


(10000, 32350)


In [44]:
from sklearn.metrics.pairwise import linear_kernel

# 10000 x 10000 서로 내적하여 코사인 유사도를 구함
# 각 항목은 두 영화의 유사도를 나타냄
cosine_sim = linear_kernel(tf_idf_matrix, tf_idf_matrix)

print(cosine_sim.shape)

(10000, 10000)


In [45]:
# 중복을 제거하여 영화 제목을 시리즈로 생성
indices = pd.Series(data.index, index=data["title"]).drop_duplicates()

print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [46]:
# 유사한 영화를 구함
def get_similar(title, indices, cosine_sim):

    # 영화의 인덱스를 구함
    try:
        index = indices[title]
    except:
        return None
    
    # 해당 영화의 유사도를 배열로 변환
    # 0 : 인덱스, 1 : 유사도
    scores = list(enumerate(cosine_sim[index]))

    # 유사도(x[1] 항목)를 기준으로 높은 순으로 정렬
    scores = sorted(scores, key=lambda x: x[1], reverse=True)

    # 가장 유사도가 높은 자신을 제외하고 5개를 추출
    scores = scores[1:6]

    # 인덱스를 구함
    indices = [x[0] for x in scores]

    # 각 인덱스의 영화 제목을 구함
    titles = data["title"].iloc[indices] 
    
    return titles

In [47]:
get_similar("Toy Story", indices, cosine_sim)


2997              Toy Story 2
8327                The Champ
1071    Rebel Without a Cause
3057          Man on the Moon
1932                Condorman
Name: title, dtype: object

In [48]:
get_similar("Star Wars", indices, cosine_sim)


1154    The Empire Strikes Back
1167         Return of the Jedi
1267               Mad Dog Time
5187        The Triumph of Love
309           The Swan Princess
Name: title, dtype: object

In [51]:
'''
어휘가 다른 문서에는 별로 등장하지 않고 특정 문서에만 집중적으로 등장할 때 그 어휘야말로 실질적으로 그 문서의 주제를 잘 담고 있는 핵심어라 할 수 있다.
'''
import pandas as pd
df = pd.read_csv('smartphone.csv', encoding='utf-8')
galexy_posts = df.get('Title')# + " " + df.get('Description')
galexy_posts

0                           셀카봉 스마트폰 삼각대 블루투스 굿
1              스마트폰을 웹캠으로 사용하기 (Driodcam/드로이드캠)
2                    스마트폰 원격시동 경보기 마이키 링크 아반떼MD
3      연이어 출시된 삼성 가성비 스마트폰 갤럭시 A31과 A51, 어떤... 
4             핸드폰 방수팩 스마트폰 방수케이스로 15M잠수도 문제 없어.
                         ...                   
236           나도써봤다 :: 엘바 스마트폰 셀카봉 삼각대 짐벌 X-PRO
237            스마트폰 용량 늘리기 알짜팁~~2분 투자해  20기가 늘림
238    PC, 스마트폰, 닌텐도 스위치 등 다양한 기기 지원! 샥스 S1... 
239              [해외 주식] 애플, 스마트폰 시장을 '또' 선도한다.
240                    스마트폰 렌즈 - APEXEL 에이펙셀 6종
Name: Title, Length: 241, dtype: object

In [54]:
from konlpy.tag import Okt
tagger = Okt()  

galexy_stop_words = "은 이 것 등 더 를 좀 즉 인 옹 때 만 원 이때 개 일 기 시 럭 갤 성 삼 스 폰 트 드 기 이 리 폴 사 전 마 자 플 블 가 중 북 수 팩 년 월 저 탭"
galexy_stop_words = galexy_stop_words.split(' ')
galexy_stop_words[0:10]

# 불용어 제거
galexy_nouns = []
for post in galexy_posts:
    for noun in tagger.nouns(post):
        if noun not in galexy_stop_words:
            galexy_nouns.append(noun)

galexy_nouns[0:10]

from collections import Counter
num_top_nouns = 10
galexy_nouns_counter = Counter(galexy_nouns)
galexy_top_nouns = dict(galexy_nouns_counter.most_common(num_top_nouns))
galexy_top_nouns

{'스마트폰': 250,
 '삼각대': 22,
 '갤럭시': 20,
 '사진': 19,
 '거치': 14,
 '삼성': 13,
 '방법': 13,
 '강의': 13,
 '살균': 12,
 '활용': 12}

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
galexy_tfidv = TfidfVectorizer(stop_words=["스마트폰"]).fit(galexy_top_nouns)
galexy_tfidv.vocabulary_

{'삼각대': 6,
 '갤럭시': 1,
 '사진': 4,
 '거치': 2,
 '삼성': 7,
 '방법': 3,
 '강의': 0,
 '살균': 5,
 '활용': 8}

In [56]:
galexy_tfidv.transform(galexy_posts).toarray()

print(pd.DataFrame(galexy_tfidv.transform(galexy_posts).toarray()))

       0         1    2    3    4    5    6         7    8
0    0.0  0.000000  0.0  0.0  0.0  0.0  1.0  0.000000  0.0
1    0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.000000  0.0
2    0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.000000  0.0
3    0.0  0.707107  0.0  0.0  0.0  0.0  0.0  0.707107  0.0
4    0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.000000  0.0
..   ...       ...  ...  ...  ...  ...  ...       ...  ...
236  0.0  0.000000  0.0  0.0  0.0  0.0  1.0  0.000000  0.0
237  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.000000  0.0
238  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.000000  0.0
239  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.000000  0.0
240  0.0  0.000000  0.0  0.0  0.0  0.0  0.0  0.000000  0.0

[241 rows x 9 columns]
